**Francisco Hernán Ruiz Cruz**

**Investigación de Operaciones**

**Optimización para la selección de cultivos**

In [ ]:
import numpy as np
from scipy.optimize import linprog
from itertools import combinations

area = float(input("Ingrese el área de su terreno en hectáreas: "))
numero = int(input("Ingrese el número de tipos de cultivos: "))
costo = input("Ingrese el costo por hectárea de cada cultivo separados por coma: ")
agua = input("Ingrese la cantidad de agua por hectárea que requiere cada cultivo durante todo su desarrollo separados por coma: ")
tiempo = input("Ingrese el tiempo en meses para cada cultivo desde la siembra hasta la cosecha separados por coma: ")
ganancia = input("Ingrese la ganancia por hectárea en pesos que cada cultivo le genera separados por coma: ")

costo_lista = costo.split(",")
agua_lista = agua.split(",")
tiempo_lista = tiempo.split(",")
ganancia_lista = ganancia.split(",")

k1 = [float(numero) for numero in costo_lista]
l1 = [float(numero) for numero in agua_lista]
t1 = [float(numero) for numero in tiempo_lista]
g1 = [float(numero) for numero in ganancia_lista]

A = area  # área total del terreno
n = numero  # número de cultivos
k = k1  # costos de cada cultivo
l = l1  # agua que necesita cada cultivo
t = t1  # tiempo que tarda cada cultivo
g = g1  # ganancias por cada cultivo
# Pesos para los objetivos
w_costo = 1
w_agua = 1
w_tiempo = 1
w_ganancia = 1 
# Función objetivo (minimizar costos, maximizar ganancias)
c = [w_costo * k[i] + w_agua * l[i] + w_tiempo * t[i] - w_ganancia * g[i] for i in range(n)]

umbral_rentabilidad = max(g) * 0.75
cultivos_rentables = [i for i in range(n) if g[i] >= umbral_rentabilidad]

if not cultivos_rentables:
    print("No hay cultivos rentables disponibles.")
else:
    n_rentables = len(cultivos_rentables)
    c_rentables = [c[i] for i in cultivos_rentables]
    A_total = [1] * n_rentables
    b = [A]
    minimo_cultivos = 2 

    restricciones_min_cultivos = []
    for comb in combinations(range(n_rentables), n_rentables - minimo_cultivos):
        restriccion = [0] * n_rentables
        for idx in comb:
            restriccion[idx] = 1
        restricciones_min_cultivos.append(restriccion)
    b_min_cultivos = [A - 0.01] * len(restricciones_min_cultivos) # Evitar que un solo cultivo reciba todo el área

    diferencia_significativa = 0.10 * A  # 10% del área total como límite de diferencia
    restricciones_diff = []
    for i in range(n_rentables):
        for j in range(i + 1, n_rentables):
            restriccion = [0] * n_rentables
            restriccion[i] = 1
            restriccion[j] = -1
            restricciones_diff.append(restriccion)
            restriccion_neg = [0] * n_rentables
            restriccion_neg[i] = -1
            restriccion_neg[j] = 1
            restricciones_diff.append(restriccion_neg)

    b_diff = [diferencia_significativa] * len(restricciones_diff)
    A_ub = restricciones_min_cultivos + restricciones_diff
    b_ub = b_min_cultivos + b_diff
    bounds = [(0, A) for _ in range(n_rentables)] #áreas no negativas y no superen el área total
    # Optimizar
    res = linprog(c_rentables, A_eq=[A_total], b_eq=b, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    if res.success:
        areas = [0] * n
         # Asignar áreas solo a los cultivos rentables 
        for i, idx in enumerate(cultivos_rentables):
            areas[idx] = int(np.round(res.x[i])) 
        # Asegurarse que la suma de las áreas no exceda el área total
        while sum(areas) > A:
            for i in range(n):
                if areas[i] > 0:
                    areas[i] -= 1
                    if sum(areas) <= A:
                        break
        margen_diferencia = 5 
        # Comprobar si hay cultivos con áreas iguales y ajustar
        cultivos_rentables = [i for i in range(n) if areas[i] > 0]
        for i in range(len(cultivos_rentables)):
            for j in range(i + 1, len(cultivos_rentables)):
                if areas[cultivos_rentables[i]] == areas[cultivos_rentables[j]] and areas[cultivos_rentables[i]] > 0:
                    areas[cultivos_rentables[j]] += margen_diferencia

        print("Solución encontrada:")
        for i, area in enumerate(areas):
            print(f"Área asignada al cultivo {i + 1}: {area} hectáreas")
    else:
        print("No se encontró una solución óptima.")
        areas = [0] * n
        